In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Load data
data = pd.read_csv('data-sets/alcohol_sales.csv')

# Convert date column to datetime object
data['date'] = pd.to_datetime(data['date'])

# Set date column as the index of the DataFrame
data = data.set_index('date')

# Split data into features and target
X = data.index.values.astype(float).reshape(-1, 1)
y = data['sales']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Assume that we have X_train, y_train, X_val, y_val as tensors

# Model Selection
class LSTMModel(nn.Module):
    def _init_(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self)._init_()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(1, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out

model = LSTMModel(input_dim=X_train.shape[2], hidden_dim=32, output_dim=1)

# Model Compilation
learning_rate = 0.001
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Model Training
num_epochs = 100
batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Model Evaluation
val_dataset = TensorDataset(X, y)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset))
with torch.no_grad():
    for inputs, targets in val_loader:
        outputs = model(inputs)
        mse = criterion(outputs, targets)
        mae = nn.L1Loss()(outputs, targets)
        rmse = torch.sqrt(mse)
        print('Mean Squared Error:', mse.item())
        print('Mean Absolute Error:', mae.item())
        print('Root Mean Squared Error:', rmse.item())

IndexError: tuple index out of range